In [1]:
import warnings
warnings.filterwarnings("ignore")
import ea_wrangle as w

import pandas as pd
import numpy as np

#splits
from sklearn.model_selection import train_test_split

#visualization
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

#scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import precision_score, accuracy_score, recall_score, classification_report

#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression


# Acquire

In [2]:
merged_data = w.left_join_csv('austin_animal_outcomes.csv', 'austin_animal_intakes.csv', 'merged_data.csv')

In [3]:
merged_data 

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197199,A884455,NaN,07/10/2023 07:52:00 AM,Jul 2023,07/03/2021,Transfer,Snr,Cat,Unknown,2 years,...,07/03/2023 11:55:00 AM,July 2023,8104 Taza Trail in Austin (TX),Stray,Normal,Cat,Unknown,2 years,Domestic Shorthair,Orange Tabby
197200,A884621,NaN,07/06/2023 06:53:00 PM,Jul 2023,07/06/2021,Transfer,Partner,Dog,Intact Male,2 years,...,07/06/2023 07:05:00 AM,July 2023,Rundberg And Ih 35 in Austin (TX),Stray,Injured,Dog,Intact Male,2 years,Pit Bull,Fawn/White
197201,A884354,NaN,07/10/2023 07:51:00 AM,Jul 2023,07/01/2020,Transfer,Snr,Cat,Intact Male,3 years,...,07/01/2023 04:06:00 PM,July 2023,1710 E Oltorf Rd Apt 814 in Austin (TX),Stray,Normal,Cat,Intact Male,3 years,Domestic Shorthair,Cream Tabby
197202,A878160,Lady,07/10/2023 10:54:00 AM,Jul 2023,04/07/2021,Transfer,Partner,Dog,Intact Female,2 years,...,04/07/2023 03:11:00 PM,April 2023,4500 East William Cannon in Austin (TX),Stray,Normal,Dog,Intact Female,2 years,Siberian Husky Mix,Gray/Black


In [4]:
df = merged_data
df.head()

,Animal ID,Name_x,DateTime_x,MonthYear_x,Date of Birth,Outcome Type,Outcome Subtype,Animal Type_x,Sex upon Outcome,Age upon Outcome,...,DateTime_y,MonthYear_y,Found Location,Intake Type,Intake Condition,Animal Type_y,Sex upon Intake,Age upon Intake,Breed_y,Color_y
0,A794011,Chunk,05/08/2019 06:20:00 PM,May 2019,05/02/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,...,05/02/2019 04:51:00 PM,May 2019,Austin (TX),Owner Surrender,Normal,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,07/18/2018 04:02:00 PM,Jul 2018,07/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,...,07/12/2018 12:46:00 PM,July 2018,7201 Levander Loop in Austin (TX),Stray,Normal,Dog,Intact Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,08/16/2020 11:38:00 AM,Aug 2020,08/16/2019,Euthanasia,NaN,Other,Unknown,1 year,...,08/16/2020 10:10:00 AM,August 2020,Armadillo Rd And Clubway Ln in Austin (TX),Wildlife,Sick,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/08/2016 11:05:00 AM,February 2016,Dove Dr And E Stassney in Austin (TX),Stray,Normal,Dog,Intact Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A720371,Moose,02/13/2016 05:59:00 PM,Feb 2016,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,...,02/15/2016 10:37:00 AM,February 2016,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff


In [5]:
df = w.get_prep_aa(df)

In [6]:
df.columns

Index(['name', 'outcome', 'dob', 'intake_type', 'intake_datetime',
       'outcome_datetime', 'intake_condition', 'intake_age', 'outcome_age',
       'species', 'found_location', 'intake_sex', 'breed', 'color'],
      dtype='object')

In [7]:
df.head()

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby


In [8]:
df.head()

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185610 entries, 0 to 197203
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   name              185610 non-null  object        
 1   outcome           185610 non-null  object        
 2   dob               185610 non-null  datetime64[ns]
 3   intake_type       185610 non-null  object        
 4   intake_datetime   185610 non-null  datetime64[ns]
 5   outcome_datetime  185610 non-null  datetime64[ns]
 6   intake_condition  185610 non-null  object        
 7   intake_age        185610 non-null  int64         
 8   outcome_age       185610 non-null  int64         
 9   species           185610 non-null  object        
 10  found_location    185610 non-null  object        
 11  intake_sex        185610 non-null  object        
 12  breed             185610 non-null  object        
 13  color             185610 non-null  object        
dtypes: d

# Prepare

In [10]:
df.columns

Index(['name', 'outcome', 'dob', 'intake_type', 'intake_datetime',
       'outcome_datetime', 'intake_condition', 'intake_age', 'outcome_age',
       'species', 'found_location', 'intake_sex', 'breed', 'color'],
      dtype='object')

pull date form 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185610 entries, 0 to 197203
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   name              185610 non-null  object        
 1   outcome           185610 non-null  object        
 2   dob               185610 non-null  datetime64[ns]
 3   intake_type       185610 non-null  object        
 4   intake_datetime   185610 non-null  datetime64[ns]
 5   outcome_datetime  185610 non-null  datetime64[ns]
 6   intake_condition  185610 non-null  object        
 7   intake_age        185610 non-null  int64         
 8   outcome_age       185610 non-null  int64         
 9   species           185610 non-null  object        
 10  found_location    185610 non-null  object        
 11  intake_sex        185610 non-null  object        
 12  breed             185610 non-null  object        
 13  color             185610 non-null  object        
dtypes: d

In [12]:
df.head()
#make intake and outcome datetimes into just the dates, make intake and outcome ages without times

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby


In [13]:
df.head(20)

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby
6,princess,adoption,2013-03-24,owner surrender,2018-06-13 12:55:00,2020-10-05 14:37:00,normal,63,91,dog,austin (tx),spayed female,chihuahua shorthair mix,brown
7,princess,adoption,2013-03-24,owner surrender,2020-10-05 11:20:00,2020-10-05 14:37:00,normal,91,91,dog,austin (tx),spayed female,chihuahua shorthair mix,brown
8,quentin,adoption,2018-03-01,owner surrender,2020-03-01 14:19:00,2020-05-06 07:59:00,normal,24,26,dog,austin (tx),neutered male,american foxhound/labrador retriever,white/brown
9,*lalo,adoption,2022-05-04,stray,2022-06-23 15:09:00,2022-07-19 15:53:00,normal,1,2,cat,10008 parkfield in austin (tx),intact male,domestic shorthair,brown tabby
10,*donatello,adoption,2014-08-01,stray,2014-10-08 16:39:00,2014-10-18 18:52:00,normal,2,2,cat,2300 waterway bnd in austin (tx),intact male,domestic shorthair mix,black


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185610 entries, 0 to 197203
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   name              185610 non-null  object        
 1   outcome           185610 non-null  object        
 2   dob               185610 non-null  datetime64[ns]
 3   intake_type       185610 non-null  object        
 4   intake_datetime   185610 non-null  datetime64[ns]
 5   outcome_datetime  185610 non-null  datetime64[ns]
 6   intake_condition  185610 non-null  object        
 7   intake_age        185610 non-null  int64         
 8   outcome_age       185610 non-null  int64         
 9   species           185610 non-null  object        
 10  found_location    185610 non-null  object        
 11  intake_sex        185610 non-null  object        
 12  breed             185610 non-null  object        
 13  color             185610 non-null  object        
dtypes: d

In [15]:
df.isnull().sum()

name                0
outcome             0
dob                 0
intake_type         0
intake_datetime     0
outcome_datetime    0
intake_condition    0
intake_age          0
outcome_age         0
species             0
found_location      0
intake_sex          0
breed               0
color               0
dtype: int64

In [16]:

#create columns, 1: mix, 2: 2 mixes, 3: purebread


In [17]:
df.intake_type.value_counts()

stray              124947
owner surrender     45457
public assist       13947
abandoned            1259
Name: intake_type, dtype: int64

In [18]:
df.outcome.value_counts()

adoption           91983
transfer           46778
return to owner    40426
euthanasia          4514
rto-adopt           1909
Name: outcome, dtype: int64

In [19]:
df.head()

,name,outcome,dob,intake_type,intake_datetime,outcome_datetime,intake_condition,intake_age,outcome_age,species,found_location,intake_sex,breed,color
0,chunk,rto-adopt,2017-05-02,owner surrender,2019-05-02 16:51:00,2019-05-08 18:20:00,normal,24,24,cat,austin (tx),neutered male,domestic shorthair mix,brown tabby/white
1,gizmo,adoption,2017-07-12,stray,2018-07-12 12:46:00,2018-07-18 16:02:00,normal,12,12,dog,7201 levander loop in austin (tx),intact male,chihuahua shorthair mix,white/brown
3,moose,adoption,2015-10-08,stray,2016-02-08 11:05:00,2016-02-13 17:59:00,normal,4,4,dog,dove dr and e stassney in austin (tx),intact male,anatol shepherd/labrador retriever,buff
4,moose,adoption,2015-10-08,owner surrender,2016-02-15 10:37:00,2016-02-13 17:59:00,normal,4,4,dog,austin (tx),neutered male,anatol shepherd/labrador retriever,buff
5,nan,transfer,2014-03-12,stray,2014-03-18 07:11:00,2014-03-18 11:47:00,nursing,0,0,cat,12034 research in austin (tx),intact male,domestic shorthair mix,orange tabby


In [20]:
df.isnull().sum()

name                0
outcome             0
dob                 0
intake_type         0
intake_datetime     0
outcome_datetime    0
intake_condition    0
intake_age          0
outcome_age         0
species             0
found_location      0
intake_sex          0
breed               0
color               0
dtype: int64

In [21]:
df.isnull().sum()

name                0
outcome             0
dob                 0
intake_type         0
intake_datetime     0
outcome_datetime    0
intake_condition    0
intake_age          0
outcome_age         0
species             0
found_location      0
intake_sex          0
breed               0
color               0
dtype: int64

In [22]:
df['mix_breeds'] = df['breed'][df['breed'].str.contains('mix', case=False)]

In [23]:
df['two_breeds'] = df['breed'][df['breed'].str.contains('/', case=False)]

In [24]:
df['pure_bred'] = df['breed'][df['breed'].str.contains('/', case=False)]

In [25]:
df['pure_bred'] = df[~df['pure_bred'].astype(str).str.contains('mix|/', case=False)]

ValueError: Columns must be same length as key

df['pure_bred'] = df[~df['pure_bred'].astype(str).str.contains('mix|/')]


In [ ]:
df.head(10)

In [ ]:
df.shape

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion

# Question and Hypothesis Statement

# Conclusion